# The Minty Pandas takes on Datatonic's Data Challenge: Our Work Flow

In this notebook, we present a chronological summary of our workflow throughout the Datatonic data challenge. It consists of four parts: 

(1) Data pre-processing

(2) Feature engineering

(3) Exploratory data analysis (EDA), data visualisation, and ordinary least squares (OLS) regression

(4) Machine learning (ML) models

# (1) Data pre-processing

## Summary of  "removing_outliers_and_null_values.ipynb"

Two datasets describing 5000 TMDB movies were imported from Kaggle: one with movie details and another with movie credits.

After reading in the two datasets as pandas dataframes, the first thing we did was to remove null values and outliers from the movie details dataset. In a nutshell, the accomplishes the following: remove columns that contain a large number of null values as well as removing rows that contain null values. The rationale behind this is that they are unreliable sources of information. 

Secondly, outliers were discarded: all values that deviated by more than three standard deviations from the mean were removed from the datset.

Finally, the updated movie details dataframe was joined together with the movie credits dataframe to produce one master dataframe - this was exported as a pickle file that was readily available for subsequent analysis.

## Summary of "strings_to_dicts.ipynb"

In the original dataset, features such as genres, keywords, and production companies were stored in raw list JSON format. To make these features more accessible, we had to transform their format: this involved converting the data type from being a string of dictionaries into a single dictionary. For example if a movie had the entry [{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}, {"id": 10749, "name": "Romance"}] under genres, this would be converted to {'id': [12, 14, 28, 10749], 'name': ['Adventure', 'Fantasy', 'Action', 'Romance']} in the updated dataframe. This was done across columns with their entries stored in JSON format, resulting in a dataframe that was easier to work with. 

Next, we created id maps that link a given id to its corresponding name. Using the above example, the id 10749 would be mapped to the genre of Romance. These id maps were crucial for subsequent one-hot encoding of a range of variables, which in turns were instrumental for determining important features in our ML models. We discuss these below. 

# (2) Feature engineering

## Summary of "one-hot-encoding.ipynb"

As a large fraction of movies were missing keywords, we first sought to supplement these by processing each movie's overview. We produced a function which breaks up the overview into individual words, and compares these to the existing keywords from our id maps. If there was a match, the word was added to that movie's keywords dictionary. This resulted in an enhancement of existing keywords for many movies, and added entries to ones which were previously lacking.

We decided to use one hot encoding to convert the following categorical features into ones suitable for our machine learning algorithms: genres, keywords, production companies, production countries, spoken languages, crew and cast. In doing so we also collected information regarding the gender of the first and second lead cast members.

To reduce the number of these categorical features in our dataset, we only kept the top 500 most frequently occurring keywords and crew members. However, for cast members we adjusted this such that we only kept actors who appeared in >10 movies. This is because it is difficult to judge an actor purely on the number of appearences, with someone like Leonardo DiCaprio not being included in the top 500 by appearances.

As we were interested in examining movies based on books we wanted to create a binary feature which would mark whether a movie was based on a book (1) or not (0). From the keywords we found that films could be based off a range of literary material: based on novel, based on young adult novel, based on comic book. In the interest of increasing our N for this feature, we pooled these three keywords under our book feature.

## Summary of "adjusting-revenue-variable.ipynb"

As we aimed to build a model which would predict a movie's success to help guide a production company's decision to greenlight production, we decided that the measure of success should be financial. Most relevant to this, were the revenue and budget features. Ideally, we would calculate a measure similar to profit (e.g. revenue-budget), or a value normalised to budget (e.g. revenue/budget). However, there were many missing budget values, likely due to the actual values being unknown/not reported, which meant these metrics could not be calculated. As removing movies with a budget value of 0 would significantly reduce the size of our dataset, we ultimately decided to use revenue as our measure of a movie's success. Budget was used as one of our predictors, and the missing values were replaced by the median budget value calculated from all movies.

The release date feature was adapted such that the release year was ignored. This resulted in a cyclical date which would allow us to retain information about seasonality. We believed that this could have valuable insights on how release of a movie might impact its success e.g. due to awards seasons, seasonal holidays etc. 

# (3) EDA, OLS regression, and data visualisation

## EDA: Summary of "Category_plots.ipynb"
To get an idea of the number of movies based on books or sequels, the abudance of these movies were plotted against the remaining movies in the dataset over time. Overall, there is a gradual but slow increase of sequels and book-based movies in the previous few decades. 
<img src="files/plots/categoryplots_releasedate_books-sequels-other.png">

Secondly, the revenue of sequels and book-based movies were plotted in a boxplot diagram and it can be observed that they display a similar range to that achieved by other movies. 
<img src="files/plots/categoryplots_revenue-boxplot_books-sequels-other.png">

Thirdly, we have plotted the estimated profit (calculated as revenue-budget), revenue, budget, and vote average for films originally identified as based on a novel, graphic novel, or young adult novel.
<img src="files/plots/category_plots_profit-budget-revenue-voteavg_boxplot_different-novel-types.png">


## Do popular movies have higher revenues? Do higher-rated movies earn more? Summary of "some_aggregation_and_trends.ipynb"

To understand the distribution of original languages, we plotted a histogram of this feature. We found that the vast majority of movies in the TMDB dataset had English as the original language. Indeed, due to the low occurence of most languages, the scale of the y-axis of the plot is logarithmic. Due to the dominance of English as the original language, we decided to remove original language as a predictor. That is, we did not consider original language to be an important predictor of success. 
<img src="files/plots/Barchart_original_language.png">

Next, we scrutinised the popularity score to get a sense of how it was distributed across the movies. According to TMDB, the popularity score for a given movie is based on number of votes for the day, number of views for the day, number of users who marked it as a "favourite" for the day, number of users who added it to their "watchlist" for the day, release date, number of total votes, previous days score (Source: https://developers.themoviedb.org/3/getting-started/popularity). While the exact formula for the calculation of popularity is not revealed, we categorised movies based on their popularity and displayed this is in a bar chart. This categorisation was linear meaning that each category was defined by equally large popularity intervals. We found that the overwhelming majority of popularity scores for this dataset were "extremely unpopular" as defined by a popularity between 0 and 20. In contrast, only a small number of movies made it into the "extremely popular" category. In hindsight, this could indicate that uniformly sized popularity categories are not the optimal way to divide movies based on popularity. Alternatively, the popularity may be negatively skewed by design: this would make sense as the popularity depends on the numbers of users that have added a movie to their watchlist, marked it as a favorite, etc. A significant limitation of using the popularity is that it is not known exactly how it is calculated based on the variables listed above. Equally, it depends on a number of factors that are already included in the dataset anyway, including release data and number of total votes. For this reason, the popularity was discarded from most of our subsequent analysis. 
<img src="files/plots/Barchart_popularity.png">

Since there are different measures of movie success, we decided it was important to investigate the correlations between revenue (our chosen measure of movie success), and other possible measures of success such as the popularity and vote average. In particular, we found a moderate correlation (Pearson correlation coefficient of 0.66) with popularity, which makes sense given one of the criteria for popularity - people would have watched it before adding it to favorites.
<img src="files/plots/Scatterplot_revenue_popularity.png">

However, the revenue did not correlate with the vote average (a measure of the rating of a movie): a correlation coefficient of 0.15 was calculated. As a result, the vote average was kept as a variable in our later analysis. If anything, the plot suggests that a normal distribution of revenue with vote average is a more accurate description of their relationship. 
<img src="files/plots/Scatterplot_revenue_vote_average.png">



## EDA: Summary of "production_company_analysis.ipynb"

Production houses are vital to the film industry and help take care of processes involved in e.g. funding, advertising, and screening. Most people are familiar with the famous production houses like 20th Century Studios, Warner Brothers, Paramount Pictures, Universal Pictures, and Lionsgate Films. We looked into the production houses involved with the different movies as well as the different production countries. First, we aggregated the performances of different production companies and production countries and compared features such as their budget, revenue, and popularity: in doing so, we produced rankings of top 5 production companies across these features. 

Secondly, we wanted to determine whether certain production houses were correlated with certain production companies - for example we expected the above mentioned production companies would correlate with the US as a production country. 
More generally, we had two variables relating to production houses: production house and production country, we wanted to determine if the two features relayed the same information. We looked at the pairwise correlations between the two: we used the one-hot encoded values for production companies and production houses. For example, the column "China" had a zero if the movie was made in China. We found that the production companies and production countries were poorly correlated to one another. This led us to include both variables in our machine learning models.

<img src="files/plots/corrmatrix_productionhouses-vs-productioncountries.png">


## OLS: Summary of "OLS_regression_and_correlations.ipynb"

In the provided TMDB dataset, there is a huge number of potential predictors, which motivated us to carry out OLS regression to help identify features which would be particularly useful for our ML models. In particular, a function was written to compute the coefficient of determination for multivariate linear regression with any number of floating point dependent variables. For single variable OLS regression, we saw that linear regression between revenue and vote count gave an r^2 value of 0.35. The value was slightly increased to 0.39 when we did multivariate regression with both vote count and budget as dependent variables. 

<img src="files/plots/multivariate_linear_regression_plot.png">

To guide our consideration of suitable success measures, for use in future analyses, we also correlated candidate success measures including budget, revenue, runtime, vote average, and vote count. The correlation matrix shows that revenue and budget are moderately correlated - in fact, our ML models later confirmed that budget was the single greatest predictor of success. We also observed that revenue was moderately correlated with the vote count, which is plausible: if more people have voted, more people have watched the movie, which explains why the revenue is higher. 





<img src="files/plots/correlation_matrix_plot.png">

# (4) ML models

# Testing different models in scitkit learn (a summary of "scikit-models.ipynb"

To implement ML models that predicted revenue, we transformed some of the numerical data for model training and used the one-hot encoded variables. The data were split into test and training datasets.


The different ML models achieved different r^2 scores, as displayed in the figure below: the elastic net method came out on top with an r^2 value of roughly 0.53, marginally prevailing over ridge regression and XGBoost. Other methods like support vector machines (SVM) and extra-trees regressor performed less well. However it should be noted that the y-axis only extends from 0.45 to 0.55, so the differences in performance between the models is not as one might think from first glance. 
<img src="files/plots/r2_scores.png">

We note that recursive feature selection (RFE) was used for the top 2 models (elastic net and ridge regression) in order to perform feature selection. First of all, ridge regression led to the following top 10 and bottom 10 features for predicting movie revenue. As mentioned earlier, the revenue was found to be moderately correlated with the revenue, which raised the suspicion that budget was an important predictor of revenue. Indeed, our feature selection in ridge regression revealed that budget was the single most important predictor of revenue. Secondly, the runtime is the second most important predictor. Perhaps more surprisingly (or unsurprisingly), the model predicts that a movie featuring Tom Cruise is much more likely to generate high revenue.  


<img src="files/plots/top10_features_ridge.png">

In contrast, some of the features least associated with high revenues include independent films, Western movies, and movies featuring Michael Shannon (c'mon he had a stellar performance as General Zod in Man of Steel, give the guy some credit!) Interestingly, many independent (indie) films are made on low budgets, so this findings is in agreement with our conclusion that budget is the biggest predictor of revenue. 

<img src="files/plots/bottom10_features_ridge.png">



In the elatic nethod methods, we find the following top 10 and bottom features: 

<img src="files/plots/top10_features_elastic.png">
Likewise, the budget comes out as being the most important predictor of success by far, the runtime again comes in on second place, and Tom Cruise makes it into the top 10 features. 
<img src="files/plots/bottom10_features_elastic.png">
Like for ridge regression, indie films comes out as being the least important predictor for revenues, once again in agreement with the finding that budget is the biggest predictor of revenues. 